In [1]:
import copy
import math
import time
import sys
"""
Dati enter dupa fiecare solutie afisata.

Presupunem ca avem costul de mutare al unui bloc egal cu indicele in alfabet, cu indicii incepănd de la 1 (care se calculează prin 1+ diferenta dintre valoarea codului ascii al literei blocului de mutat si codul ascii al literei "a" ) . Astfel A* are trebui sa prefere drumurile in care se muta intai blocurile cu infoutputmatie mai mica lexicografic pentru a ajunge la una dintre starile scop
"""


'\nDati enter dupa fiecare solutie afisata.\n\nPresupunem ca avem costul de mutare al unui bloc egal cu indicele in alfabet, cu indicii incepănd de la 1 (care se calculează prin 1+ diferenta dintre valoarea codului ascii al literei blocului de mutat si codul ascii al literei "a" ) . Astfel A* are trebui sa prefere drumurile in care se muta intai blocurile cu infoutputmatie mai mica lexicografic pentru a ajunge la una dintre starile scop\n'

In [2]:
#informatii despre un nod din arborele de parcurgere (nu din graful initial)
class NodParcurgere:
    def __init__(self, info, parinte, cost=0, h=0):
        self.info=info
        self.parinte=parinte #parintele din arborele de parcurgere
        self.g=cost #consider cost=1 pentru o mutare
        self.h=h
        self.f=self.g+self.h

    def obtineDrum(self):
        l=[self]
        nod=self
        while nod.parinte is not None:
            l.insert(0, nod.parinte)
            nod=nod.parinte
        return l
        
    def afisDrum(self, afisCost=False, afisLung=False): #returneaza si lungimea drumului
        l=self.obtineDrum()
        
        for index,nod in enumerate(l):
            inaltimeMax=0
            for stiva in nod.info:
                if (len(stiva))>inaltimeMax:
                    inaltimeMax=len(stiva)
            print(index,")",sep='')
            for i in reversed(range(inaltimeMax)):
                for stiva in nod.info:
                    if len(stiva)>i:
                        if int(stiva[i])<10:
                            print("[",stiva[i],"]",sep='',end = '    ')
                        elif int(stiva[i])<100:
                            print("[",stiva[i],"]",sep='',end = '   ')
                        else:
                            print("[",stiva[i],"]",sep='',end = '  ')
                    else:
                        print("       ",sep='',end = '')
                print('')
            print('==============================\n')
            
        if afisCost:
            print("Cost: ", self.g)
        if afisCost:
            print("Lungime: ", len(l))
        return len(l)
    
    


    def contineInDrum(self, infoNodNou):
        nodDrum=self
        while nodDrum is not None:
            if(infoNodNou==nodDrum.info):
                return True
            nodDrum=nodDrum.parinte
        
        return False
        
    def __repr__(self):
        sir=""        
        sir+=str(self.info)
        return(sir)


    #euristica banală: daca nu e stare scop, returnez 1, altfel 0

    
    def __str__(self):
        sir=""
        maxInalt=max([len(stiva) for stiva in self.info])
        for inalt in range(maxInalt, 0, -1):
            for stiva in self.info:
                if len(stiva)< inalt:
                    sir+="  "
                else:
                    sir+=stiva[inalt-1]+" "
            sir+="\n"
        sir+="-"*(2*len(self.info)-1)
        return sir

In [3]:
class Graph: #graful problemei
    def __init__(self):

        def obtineStive(sir):
            stiveSiruri=sir.strip().split("\n") 
            listaStive=[sirStiva.strip().split(";") if sirStiva !="~" else [] for sirStiva in stiveSiruri]
            nr = 0
            for stiva in listaStive:
                for elem in stiva:
                    nr+=1
                    if not elem.isnumeric():
                        print("EROARE: Trebuie sa avem doar numere naturale")
                
            
            #validari
            return listaStive, nr
        
        
  
        
    
        while True:            
            try:
                pathCond=0
                while pathCond==0:
                    path_fisier=input("Path pentru fisier .txt: ")
                    path_fisier=path_fisier.strip()
                    if path_fisier[-4:]=='.txt':
                        pathCond=1
                    else:
                        print("\nEroare: Fisierul nu este .txt!")

                f = open(path_fisier, 'r')
            except:
                print("\nEroare: Fisierul nu exista sau path-ul este gresit!")
            else:
                break
      

        continutFisier=f.read() #citesc tot continutul fisierului
        
        
        siruriStari=continutFisier.split("-----\n")
        self.start,nr=obtineStive(siruriStari[0]) #[[1], [12,3],[5]]
        self.scop=siruriStari[1].split(" ")
        
            
        print("Stare Initiala:", self.start)
        print("Stare scop:",self.scop)
    
        #validari/optimizari
        sum = 0
        for k in self.scop:
            sum = sum + int(k)
        if nr != sum:
            print("ERRORE: Date de intrare gresite!")
            sys.exit()
            
    
    def testeaza_scop(self, nodCurent):
        k=1
        for i, stiva in enumerate(nodCurent.info):
            if len(stiva) != int(self.scop[i]):
                k = 0
        return k

    #va genera succesorii sub forma de noduri in arborele de parcurgere    


    def genereazaSuccesori(self, nodCurent, tip_euristica="euristica banala"):
        listaSuccesori=[]
        stive_c=nodCurent.info # stivele din nodul curent
        nr_stive=len(stive_c)
        for idx in range(nr_stive):#idx= indicele stivei de pe care iau bloc
            
            if len(stive_c[idx])==0 :
                continue
            copie_interm=copy.deepcopy(stive_c)
            bloc=copie_interm[idx].pop() #iau varful stivei
            
            
            for j in range(nr_stive): #j = indicele stivei pe care pun blocul 
                if idx == j: # nu punem blocul de unde l-am luat
                    continue
                
                
                #conditii de mutat aici 
                #math.gcd(a,b) - determina daca a si b sunt prime intre ele
                
                inaltimeNoua=len(copie_interm[j])
                
                if copie_interm[j]:
                    if math.gcd(int(bloc),int(copie_interm[j][-1])) != 1:
                        continue  
                    
                
                nuEgale=1
                if j!=0 and j!=nr_stive-1:
                
                    if len(copie_interm[j-1])<=inaltimeNoua and len(copie_interm[j+1])<=inaltimeNoua: 
                        nuEgale = 1
                        
                    elif len(copie_interm[j-1])>inaltimeNoua and len(copie_interm[j+1])>inaltimeNoua:
                        if int(copie_interm[j-1][inaltimeNoua])==int(bloc) or int(copie_interm[j+1][inaltimeNoua])==int(bloc):
                            nuEgale = 0
                    elif len(copie_interm[j-1])>inaltimeNoua:
                        if int(copie_interm[j-1][inaltimeNoua])==int(bloc):
                            nuegal = 0
                    elif len(copie_interm[j+1])>inaltimeNoua:
                        if int(copie_interm[j+1][inaltimeNoua])==int(bloc):
                            nuegal = 0
                elif j==0:
                    if len(copie_interm[j+1])>inaltimeNoua:
                        if int(copie_interm[j+1][inaltimeNoua])==int(bloc):
                            nuEgale = 0
                elif j==nr_stive-1:
                    if len(copie_interm[j-1])>inaltimeNoua:
                        if int(copie_interm[j-1][inaltimeNoua])==int(bloc):
                            nuEgale = 0
                if nuEgale==0:
                    continue
                
                
                
                
                stive_n=copy.deepcopy(copie_interm)#lista noua de stive
                stive_n[j].append(bloc) # pun blocul in stiva aleasa
                
                
                costMutareBloc=int(bloc)
                #blocul contine doar numarul 
                
                if not nodCurent.contineInDrum(stive_n):
                    nod_nou=NodParcurgere(stive_n,nodCurent, cost=nodCurent.g+costMutareBloc,h= self.calculeaza_h(stive_n, tip_euristica))
                    listaSuccesori.append(nod_nou)
        
#         print(str(nodCurent.info))
#         print("succesori : ")
#         for s in listaSuccesori:
#             print(s.info)
       
        return listaSuccesori


    # euristica banala
    def calculeaza_h(self, infoNod, tip_euristica="euristica banala"):
#         if tip_euristica=="euristica banala":
#             if inf not in self.scop:
#                 return 1 #se pune costul minim pe o mutare
#             return 0
        if tip_euristica=="euristica banala":
            k=1
            for i, stiva in enumerate(infoNod):
                if len(stiva) != int(self.scop[i]):
                    k = 0
            if k == 0:
                return 1
            return 0

        
        elif tip_euristica=="euristica admisibila 1":
            
            #pt fiecare stiva care nu a ajuns la o configurare scop, adunam 1
            h=0
            for iStiva, stiva in enumerate(infoNod):
                    if len(stiva) == int(self.scop[iStiva]):
                        h = h + 1
            return h
        elif tip_euristica=="euristica admisibila 2":
            
            #aduna diferenta dintre nr de blocuri curente
            #si numarul de blocuri scop
            #la h
            h=0
            for iStiva, stiva in enumerate(infoNod):
                h = h + 1 +abs(len(stiva)-int(self.scop[iStiva]))
            return h
        else: #tip_euristica=="euristica neadmisibila"
            h=0
            for iStiva, stiva in enumerate(infoNod):
                    if len(stiva) == int(self.scop[iStiva]):
                        h = 5*(h + 10)
            return h

    def __repr__(self):
        sir=""
        for (k,v) in self.__dict__.items() :
            sir+="{} = {}\n".format(k,v)
        return(sir)

In [4]:
def breadth_first(gr, nrSolutiiCautate):

    #in coada vom avea doar noduri de tip NodParcurgere (nodurile din arborele de parcurgere)
    c=[NodParcurgere(gr.start, None)]
    
    nodMaxim=0
    
    while len(c)>0:
        
        ti=time.time()
        
        if(ti-t1>120):
            print("Breadth_first a depasit timpul alocat de 2 minute!")
            return
        
        nodCurent=c.pop(0)
        if len(nodCurent.obtineDrum())>nodMaxim:
            nodMaxim=len(nodCurent.obtineDrum())

        if gr.testeaza_scop(nodCurent):
            t2=time.time()
            print("Solutie: \nTimp pana la solutie: " ,t2-t1,"secunde\n")
            nodCurent.afisDrum(afisCost=True, afisLung=True)
            print("Numarul maxim de noduri existente: ",nodMaxim)
            nodMaxim=0
            
            print("\n----------------\n")
            
            nrSolutiiCautate-=1
            
            if nrSolutiiCautate==0:
                return
            
        lSuccesori=gr.genereazaSuccesori(nodCurent)    
        c.extend(lSuccesori)

In [5]:
def a_star(gr, nrSolutiiCautate, tip_euristica):
    #in coada vom avea doar noduri de tip NodParcurgere (nodurile din arborele de parcurgere)
    
    c=[NodParcurgere(gr.start, None, 0, gr.calculeaza_h(gr.start))]
    
    nodMaxim=0
    
    while len(c)>0:
        
        ti=time.time()
        if(ti-t1>120):
            print("A_Star a depasit timpul alocat de 2 minute!",file=foutput)
            return
        
        nodCurent=c.pop(0)
        
        if len(nodCurent.obtineDrum())>nodMaxim:
            nodMaxim=len(nodCurent.obtineDrum())
        
        if gr.testeaza_scop(nodCurent):
            t2=time.time()
            print("Solutie: \nTimp pana la solutie: " ,t2-t1,"secunde\n",file=foutput)
            
            nodCurent.afisDrum(afisCost=True, afisLung=True)
            
            print("Numarul maxim de noduri existente: ",nodMaxim,file=foutput)
            nodMaxim=0
            print("\n----------------\n",file=foutput)
            
            nrSolutiiCautate-=1
            if nrSolutiiCautate==0:
                return
            
        lSuccesori=gr.genereazaSuccesori(nodCurent,tip_euristica=tip_euristica)    
        for s in lSuccesori:
            i=0
            gasit_loc=False
            for i in range(len(c)):
                #diferenta fata de UCS e ca ordonez dupa f
                if c[i].f>=s.f :
                    gasit_loc=True
                    break
            if gasit_loc:
                c.insert(i,s)
            else:
                c.append(s)


In [6]:
def depth_first(gr, nrSolutiiCautate=1):
    #vom simula o stiva prin relatia de parinte a nodului curent    
    nodMaxim=0
    df(NodParcurgere(gr.start, None), nrSolutiiCautate, nodMaxim)

                
def df(nodCurent, nrSolutiiCautate, nodMaxim):
    ti=time.time()
    if(ti-t1>120):
        print("Depth_first a depasit timpul alocat de 2 minute!",file=foutput)
        return
        
    if nrSolutiiCautate<=0: #testul acesta s-ar valida doar daca in apelul initial avem df(start,if nrSolutiiCautate=0)
        return nrSolutiiCautate
    
    if len(nodCurent.obtineDrum())>nodMaxim:
        nodMaxim=len(nodCurent.obtineDrum())
    
    if gr.testeaza_scop(nodCurent):
        
        t2=time.time()
        print("Solutie: \nTimp pana la solutie: ",t2-t1,"secunde\n",file=foutput)
        
        nodCurent.afisDrum()
        print("Numarul max de noduri existente:", nodMaxim,file=foutput)
        nodMaxim=0
        print("\n----------------\n",file=foutput)
        
        nrSolutiiCautate-=1
        if nrSolutiiCautate==0:
            return nrSolutiiCautate
    lSuccesori=gr.genereazaSuccesori(nodCurent)    
    
    for sc in lSuccesori:
        if nrSolutiiCautate!=0:
            
            nrSolutiiCautate=df(sc, nrSolutiiCautate,nodMaxim)
    
    return nrSolutiiCautate
#df(a)->df(b)->df(c)->df(f)
#############################################


In [7]:
def a_star_opt(gr):
    #in coada vom avea doar noduri de tip NodParcurgere (nodurile din arborele de parcurgere)
    l_open=[NodParcurgere(gr.start, None, 0, gr.calculeaza_h(gr.start))]
    #l_open contine nodurile candidate pentru expandare (este echivalentul lui c din A* varianta neoptimizata)
    #l_closed contine nodurile expandate
    l_closed=[]
    
    nodMaxim=0
    
    while len(l_open)>0:
        
        ti=time.time()
        if(ti-t1>120):
            print("A_Star_Optimizat a depasit timpul alocat de 2 minute!",file=foutput)
            return
        
        nodCurent=l_open.pop(0)
        #print("\nNOD CURENT:\n",nodCurent.afisDrum())
        l_closed.append(nodCurent)
        
        if len(nodCurent.obtineDrum())>nodMaxim:
            nodMaxim=len(nodCurent.obtineDrum())
        
        if gr.testeaza_scop(nodCurent):
            t2=time.time()
            print("Solutie: \nTimp pana la solutie: ",t2-t1,"s\n",file=foutput)
            
            nodCurent.afisDrum(afisCost=True, afisLung=True)
            print("Numarul max de noduri existente:", nodMaxim,file=foutput)
            nodMaxim=0
            
            print("\n----------------\n",file=foutput)
            return
        lSuccesori=gr.genereazaSuccesori(nodCurent)    
        for s in lSuccesori:
            gasitC=False
            for nodC in l_open:
                if s.info==nodC.info:
                    gasitC=True
                    if s.f>=nodC.f:
                        lSuccesori.remove(s)
                    else:#s.f<nodC.f
                        l_open.remove(nodC)
                    break
            if not gasitC:
                for nodC in l_closed:
                    if s.info==nodC.info:
                        if s.f>=nodC.f:
                            lSuccesori.remove(s)
                        else:#s.f<nodC.f
                            l_closed.remove(nodC)
                        break
        for s in lSuccesori:
            i=0
            gasit_loc=False
            for i in range(len(l_open)):
                #diferenta fata de UCS e ca ordonez crescator dupa f
                #daca f-urile sunt egale ordonez descrescator dupa g
                if l_open[i].f>s.f or (l_open[i].f==s.f and l_open[i].g<=s.g) :
                    gasit_loc=True
                    break
            if gasit_loc:
                l_open.insert(i,s)
            else:
                l_open.append(s)

In [8]:
def ida_star(gr, nrSolutiiCautate):
    
    nodStart=NodParcurgere(gr.start, None, 0, gr.calculeaza_h(gr.start))
    limita=nodStart.f
    
    nodMaxim=0
    while True:
        ti=time.time()
        
        if(ti-t1>120):
            print("Ida_Star a depasit timpul alocat de 2 minute!",file=foutput)
            return

        #print("Limita de pornire: ", limita)
        nrSolutiiCautate, rez= construieste_drum(gr, nodStart, limita,nrSolutiiCautate, nodMaxim)
        if rez=="gata":
            break
        if rez==float('inf'):
            print("Nu mai exista solutii!",file=foutput)
            break
        limita=rez
        #print(">>> Limita noua: ", limita)
        #input()


def construieste_drum(gr, nodCurent, limita, nrSolutiiCautate, nodMaxim):
    #print("A ajuns la: ", nodCurent)
    if nodCurent.f>limita:
        return nrSolutiiCautate, nodCurent.f
    
    if len(nodCurent.obtineDrum())>nodMaxim:
            nodMaxim=len(nodCurent.obtineDrum())
    
    if gr.testeaza_scop(nodCurent) and nodCurent.f==limita :
        t2=time.time()
        
        print("Solutie: \nTimp pana la solutie: ",t2-t1,"s\n",file=foutput)
        
        nodCurent.afisDrum(afisCost=True, afisLung=True)
        #print(limita)
        print("Numarul max de noduri existente:", nodMaxim,file=foutput)
        nodMaxim=0
        print("\n----------------\n",file=foutput)
        
        nrSolutiiCautate-=1
        if nrSolutiiCautate==0:
            return 0,"gata"
    lSuccesori=gr.genereazaSuccesori(nodCurent)    
    minim=float('inf')
    for s in lSuccesori:
        nrSolutiiCautate, rez=construieste_drum(gr, s, limita, nrSolutiiCautate,nodMaxim)
        if rez=="gata":
            return 0,"gata"
        #print("Compara ", rez, " cu ", minim)
        if rez<minim:
            minim=rez
            #print("Noul minim: ", minim)
    return nrSolutiiCautate, minim

In [9]:
def uniform_cost(gr, nrSolutiiCautate=1):

    #in coada vom avea doar noduri de tip NodParcurgere (nodurile din arborele de parcurgere)
    c=[NodParcurgere(gr.start, None, 0)]
    
    nodMaxim=0
    while len(c)>0:
        ti=time.time()
        
        if(ti-t1>120):
            print("Uniform_Cost a depasit timpul alocat de 2 minute!",file=foutput)
            return
        
        #print("Coada actuala: " + str(c))
        #input()
        nodCurent=c.pop(0)
        
        if len(nodCurent.obtineDrum())>nodMaxim:
            nodMaxim=len(nodCurent.obtineDrum())
        
        if gr.testeaza_scop(nodCurent):
            t2=time.time()
            print("Solutie: \nTimp pana la solutie: ",t2-t1,"s\n",file=foutput)
            
            nodCurent.afisDrum(afisCost=True, afisLung=True)
            print("Numarul max de noduri existente:", nodMaxim,file=foutput)
            nodMaxim=0
            
            print("\n----------------\n",file=foutput)
            
            nrSolutiiCautate-=1
            if nrSolutiiCautate==0:
                return
        lSuccesori=gr.genereazaSuccesori(nodCurent)    
        for s in lSuccesori:
            i=0
            gasit_loc=False
            for i in range(len(c)):
                #ordonez dupa cost(notat cu g aici și în desenele de pe site)
                if c[i].g>s.g :
                    gasit_loc=True
                    break
            if gasit_loc:
                c.insert(i,s)
            else:
                c.append(s)

In [10]:


gr=Graph()                

# while True:
#     try:
#         pathCond=0
#         while pathCond==0:
#             output_path=input("Path pentru fisier de output.txt: ")
#             output_path=output_path.strip()
#             if output_path[-4:]=='.txt':
#                 pathCond=1
#             else:
#                 print("\nEroare: Fisierul nu este .txt!")

#         foutput = open(output_path, 'w')
#     except:
#         print("\nEroare: Fisierul nu exista sau path-ul este gresit!")
#     else:
#         break
# print("Verificati fisierul output.txt")


print("Solutii obtinute cu breadth first:")
t1=time.time()
breadth_first(gr, nrSolutiiCautate=1)

# print("\n\n\nURMATOAREA METODA foLOSTIA:\n\n\n",file=foutput)

# print("Solutii obtinute cu Depth_First:",file=foutput)
# t1=time.time()
# depth_first(gr, nrSolutiiCautate=1)

# print("\n\n\nURMATOAREA METODA foLOSTIA:\n\n\n",file=foutput)

# print("Solutii obtinute cu A_Star:",file=foutput)
# t1=time.time()
# a_star(gr, nrSolutiiCautate=1, tip_euristica="euristica banala",file=foutput)

# print("\n\n\nURMATOAREA METODA foLOSTIA:\n\n\n",file=foutput)

# print("Solutii obtinute cu A_Star_Optimizat:",file=foutput)
# t1=time.time()
# a_star_opt(gr)

# print("\n\n\nURMATOAREA METODA foLOSTIA:\n\n\n",file=foutput)

# print("Solutii obtinute cu Ida_Star:",file=foutput)
# t1=time.time()
# ida_star(gr, nrSolutiiCautate=1)

# print("\n\n\nURMATOAREA METODA foLOSTIA:\n\n\n",file=foutput)

# print("Solutii obtinute cu Uniform_cost:",file=foutput)
# t1=time.time()
# uniform_cost(gr, nrSolutiiCautate=1)




#C:\Users\Dragos\TEMA_CAUTARE_KR\input.txt
#C:\Users\Dragos\TEMA_CAUTARE_KR\output.txt

Path pentru fisier .txt: C:\Users\Dragos\TEMA_CAUTARE_KR\input.txt
Stare Initiala: [['21', '55'], ['15', '7', '210', '9'], ['30', '50'], ['22', '18', '25']]
Stare scop: ['5', '5', '0', '1']
Solutii obtinute cu breadth first:
Solutie: 
Timp pana la solutie:  0.8484635353088379 secunde

0)
       [9]                  
       [210]         [25]   
[55]   [7]    [50]   [18]   
[21]   [15]   [30]   [22]   

1)
       [25]                 
       [9]                  
       [210]                
[55]   [7]    [50]   [18]   
[21]   [15]   [30]   [22]   

2)
       [25]                 
       [9]                  
[18]   [210]                
[55]   [7]    [50]          
[21]   [15]   [30]   [22]   

3)
[25]   [9]                  
[18]   [210]                
[55]   [7]    [50]          
[21]   [15]   [30]   [22]   

4)
       [50]                 
[25]   [9]                  
[18]   [210]                
[55]   [7]                  
[21]   [15]   [30]   [22]   

5)
[22]   [50]             